# Library

In [24]:
from tqdm import tqdm
import numpy as np
import re

In [3]:
import pandas as pd
import modin.pandas as mpd

In [4]:
import ray
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

2022-11-25 15:35:09,897	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.9.13
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8266


## data load

In [10]:
col = pd.read_table('data/컬럼명.txt', sep=',')
col

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,배기량,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,제원승인일자,구동형식,변속기종류


In [11]:
col.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

In [20]:
df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col.columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528586 entries, 0 to 1528585
Data columns (total 40 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   차량관리번호       1528586 non-null  object 
 1   차량등록번호       1528586 non-null  object 
 2   본거지법정동코드     1528586 non-null  int64  
 3   소유자구분        1528586 non-null  object 
 4   차종_등록정보      1528584 non-null  object 
 5   용도           1528586 non-null  object 
 6   차대번호         1528586 non-null  object 
 7   최초등록일자       1528586 non-null  int64  
 8   차량연식         1528586 non-null  int64  
 9   차량제작일자       1358925 non-null  float64
 10  검사유효일        1528574 non-null  float64
 11  제원관리번호       1528586 non-null  object 
 12  배출가스인증번호     1405585 non-null  object 
 13  배출가스등급       1528586 non-null  int64  
 14  저공해조치종류      195 non-null      object 
 15  저공해조치구조변경YN  1528586 non-null  object 
 16  차량말소YN       1528586 non-null  object 
 17  취득일자         1435514 non-null  object 
 18  취득

In [21]:
df.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,배기량,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,제원승인일자,구동형식,변속기종류
0,KMFZCY7JBCU784032-01,97구4490,1144010700,법인,화물,자가용,KMFZCY7JBCU784032,20110817,2012,20110811.0,...,2497.0,6,5120.0,1740.0,1965.0,대한민국,N,20110524,NaN,자동
1,KMHJN81VP7U578447-01,11너2154,2650010100,주민,승용,자가용,KMHJN81VP7U578447,20070102,2007,20061104.0,...,1991.0,5,4325.0,1800.0,1680.0,대한민국,N,20060621,NaN,자동
2,KPACE4AB1AP070600-01,96어5714,4729025024,주민,화물,자가용,KPACE4AB1AP070600,20091221,2010,20091210.0,...,1998.0,5,4965.0,1900.0,1755.0,대한민국,N,20090513,NaN,자동
3,KMHNU81WP7U009391-01,11가1154,4117110100,사업자,승용,자가용,KMHNU81WP7U009391,20070212,2007,20070207.0,...,2959.0,7,4840.0,1970.0,1805.0,대한민국,N,20060821,NaN,자동
4,KMHDT41TP9U638425-01,22보7099,4711331026,주민,승용,자가용,KMHDT41TP9U638425,20081128,2009,20081123.0,...,1582.0,5,4505.0,1775.0,1480.0,대한민국,N,20080729,NaN,수동


### [e]차량관리번호
- 오류(영문과 숫자가 아닌 값) : 7038개

In [23]:
df.iloc[:, 0].value_counts()

KMFZCY7JBCU784032-01    1
KMHSH81VP9U467470-01    1
KNAGA2263NA273801-01    1
KNCSJX74BBK577862-01    1
KNADG414AAS597155-01    1
                       ..
KNAFA2223PA131127-01    1
KPACA4AB1BP099521-01    1
KNCSJZ74BBK571321-01    1
KNAGA2223MA251640-01    1
KNME5C2MM8P008790-01    1
Name: 차량관리번호, Length: 1528586, dtype: int64

In [38]:
count = 0
p0 = re.compile('\w+[-]\d+')
idx0 = []
for i, one in tqdm(enumerate(df.iloc[:, 0].to_numpy())):
    if not p0.search(one):
        idx0.append(i)
        count += 1
count

1528586it [00:01, 1026573.24it/s]


7038

In [39]:
idx1[:5]

[396, 586, 700, 821, 1227]

In [40]:
df[['차량관리번호', '차량등록번호']].reindex(idx0)

,차량관리번호,차량등록번호
396,1199서울4브2371,서울4브2371
586,4599전북1우1923,전북1우1923
700,1199서울1즈5001,서울1즈5001
821,2799대구4수5818,대구4수5818
1227,4199경기59너9481,경기59너9481
...,...,...
1527680,1199서울52마5593,서울52마5593
1527890,4199경기3노8469,경기3노8469
1528062,3099대전1므5676,대전1므5676
1528409,1199서울2두4957,서울2두4957


### [e]차량등록번호
- 오류
    - [한글][숫자][한글][숫자] 형식에 맞지 않는 값 : 1,211,007개
    - [숫자][한글][숫자] 형식에 맞지 않는 값 : 0개

In [41]:
df.iloc[:, 1].value_counts()

97구4490      1
27나1278      1
인천1더9940     1
83두1780      1
12소2340      1
            ..
서울1러2561     1
95마1880      1
87마2994      1
경기45마7935    1
55고3109      1
Name: 차량등록번호, Length: 1528586, dtype: int64

In [42]:
count = 0
p1 = re.compile('\d+[가-힣]+')
idx1 = []
for i, one in tqdm(enumerate(df.iloc[:, 1].to_numpy())):
    if not p1.search(one):
        idx1.append(i)
        count += 1
count

1528586it [00:01, 1048910.52it/s]


0

In [43]:
count = 0
p11 = re.compile('[가-힣]+\d+[가-힣]+\d+')
idx11 = []
for i, one in tqdm(enumerate(df.iloc[:, 1].to_numpy())):
    if not p11.search(one):
        idx11.append(i)
        count += 1
count

1528586it [00:01, 822960.39it/s]


1211007

In [44]:
count = 0
p12 = re.compile('\d+[가-힣]+\d+')
idx12 = []
for i, one in tqdm(enumerate(df.iloc[:, 1].to_numpy())):
    if not p12.search(one):
        idx12.append(i)
        count += 1
count

1528586it [00:01, 1077454.89it/s]


0

### 본거지법정동코드
- 오류
    - 숫자가 아닌 값 : 0개
    - 길이가 10이 아닌 값 : 0개

In [37]:
df.iloc[:, 2].value_counts()

4139013200    5442
1150010300    4246
1162010200    3960
2817710500    3936
1135010500    3513
              ... 
4479039025       1
4575039000       1
4119710400       1
4163055000       1
4371025030       1
Name: 본거지법정동코드, Length: 20119, dtype: int64

In [46]:
count = 0
p2 = re.compile('\d+')
idx2 = []
for i, one in tqdm(enumerate(df.iloc[:, 2].to_numpy())):
    if not p2.search(str(one)):
        idx2.append(i)
        count += 1
count

1528586it [00:02, 612922.49it/s]


0

In [67]:
count = 0
idx21 = []
for i, one in tqdm(enumerate(df.iloc[:, 2].to_numpy())):
    if len(str(one)) != 10:
        idx21.append(i)
        count += 1
count

1528586it [00:01, 816483.62it/s]


0

### 소유자구분
- 오류 : 0개

In [47]:
df.iloc[:, 3].value_counts()

주민     1320642
법인      139402
사업자      42860
외국인      21568
기타        4114
Name: 소유자구분, dtype: int64

### 차종_등록정보

In [48]:
df.iloc[:, 4].value_counts()

승용    850752
화물    520825
승합    148852
특수      8155
Name: 차종_등록정보, dtype: int64

### 용도

In [49]:
df.iloc[:, 5].value_counts()

자가용    1481507
영업용      41362
관용        5717
Name: 용도, dtype: int64

### [e]차대번호
- 오류
    - 1\. 영문과 숫자가 아닌 값 : 0개
    - 2\. 길이가 17자리가 아닌 값 : 2,630개
    - 3\. 숫자만 있는 값 : 2,574개
        - 위 두 조건사이에 겹치지 않는 값(조건2와 조건3의 XOR(대칭차집합)) : 58개
        - 위 두 조건모두 만족하는 값(조건2와 조건3의 교집합) : 2,573개

In [57]:
len('KMHCF21DPSU000012')

17

In [50]:
df.iloc[:, 6].value_counts()

KMHCF21DPSU000012    5
KLAVM69H1JB075787    2
KMHMF21FPPU018261    2
KMHVF21JPKU161128    2
KLY7T51BBNC011472    2
                    ..
KMFZCZ7JABU656518    1
KNAFA2223PA131127    1
KPACA4AB1BP099521    1
KNCSJZ74BBK571321    1
KNME5C2MM8P008790    1
Name: 차대번호, Length: 1528564, dtype: int64

In [51]:
count = 0
p6 = re.compile('\w+')
idx6 = []
for i, one in tqdm(enumerate(df.iloc[:, 6].to_numpy())):
    if not p6.search(one):
        idx6.append(i)
        count += 1
count

1528586it [00:01, 1034382.21it/s]


0

In [58]:
count = 0
idx61 = []
for i, one in tqdm(enumerate(df.iloc[:, 6].to_numpy())):
    if len(one) != 17:
        idx61.append(i)
        count += 1
count

1528586it [00:00, 1966905.41it/s]


2630

In [59]:
df.iloc[:, 6].reindex(idx61)[:5]

429       119811326107
692     27931130152305
1320      119111327184
1475      119811229792
1544      219211301625
Name: 차대번호, dtype: object

In [60]:
count = 0
p62 = re.compile('[a-zA-Z]+')
idx62 = []
for i, one in tqdm(enumerate(df.iloc[:, 6].to_numpy())):
    if not p62.search(one):
        idx62.append(i)
        count += 1
count

1528586it [00:01, 1181152.42it/s]


2574

In [61]:
df.iloc[:, 6].reindex(idx62)[:5]

429       119811326107
692     27931130152305
1320      119111327184
1475      119811229792
1544      219211301625
Name: 차대번호, dtype: object

In [66]:
inter_idx61_62 = list(set(idx61) & set(idx62))
len(inter_idx61_62)

2573

In [63]:
diff_idx61_62 = list(set(idx61) ^ set(idx62))
len(diff_idx61_62)

58

In [64]:
df.iloc[:, 6].reindex(diff_idx61_62)[:5]

278554     KN3HN8D3XK094667
1343562    KNAGA2222HS01099
360700     KLATF08G1JB08216
303394           EU13315046
1499532    KMHVF31JPKU06672
Name: 차대번호, dtype: object

In [65]:
df.iloc[:, 6].reindex(diff_idx61_62)[-5:]

1121413    KMBF32S4LU186819
498940          018013212A7
1228141     KNAGB22KS203406
1359226    KNME5C2MM8P00423
196218     KNAD2322JS055503
Name: 차대번호, dtype: object

### [e]최초등록일자
- 오류
    - 숫자가 아닌 값 : 0개
    - 길이가 8이 아닌 값 : 2개

In [52]:
df.iloc[:, 7].value_counts()

20080102    4651
20110103    4609
20090102    4263
20080103    2707
20100104    2634
            ... 
20160127       1
20130617       1
20040716       1
20130712       1
20220614       1
Name: 최초등록일자, Length: 8446, dtype: int64

In [54]:
count = 0
p7 = re.compile('\d+')
idx7 = []
for i, one in tqdm(enumerate(df.iloc[:, 7].to_numpy())):
    if not p7.search(str(one)):
        idx7.append(i)
        count += 1
count

1528586it [00:02, 623283.23it/s]


0

In [55]:
count = 0
idx71 = []
for i, one in tqdm(enumerate(df.iloc[:, 7].to_numpy())):
    if len(str(one)) != 8:
        idx71.append(i)
        count += 1
count

1528586it [00:01, 788935.48it/s]


2

In [56]:
df.iloc[:, 7].reindex(idx71)

110669       1993
489146    9960617
Name: 최초등록일자, dtype: int64

### 차량연식
- 오류
    - 길이 4 아닌 값 : 0개
    - 숫자가 아닌 값 : 0개

In [68]:
df.iloc[:, 8].value_counts()

2010    252711
2011    237575
2008    226868
2009    213544
2007    107029
2012     94187
1994     37783
1996     36266
1995     35395
1993     34281
2006     34010
1992     33927
1991     30328
1997     29988
1990     26704
1989     20689
1999     20030
2000     13327
1988     11854
1998     10236
2002      8971
2001      7836
1987      1278
2005      1068
2017       732
2018       483
2016       482
2015       463
2004       214
2003       169
2013        98
2014        13
2019        13
1986        12
1985         4
2020         3
1984         3
1979         3
1983         3
1978         1
1981         1
1980         1
1901         1
1968         1
1973         1
Name: 차량연식, dtype: int64

In [69]:
count = 0
idx8 = []
for i, one in tqdm(enumerate(df.iloc[:, 8].to_numpy())):
    if len(str(one)) != 4:
        idx8.append(i)
        count += 1
count

1528586it [00:01, 822583.55it/s]


0

In [70]:
count = 0
p81 = re.compile('\d+')
idx81 = []
for i, one in tqdm(enumerate(df.iloc[:, 8].to_numpy())):
    if not p81.search(str(one)):
        idx81.append(i)
        count += 1
count

1528586it [00:02, 627727.06it/s]


0

### 차량제작일자
- 오류
    - 8자리 아닌 값 : 4개
    - nan 값 : 169,661개
    - 숫자가 아닌 값(nan값 제외) : 0개

In [77]:
df.iloc[:, 9].value_counts()

20090629.0    1459
20090612.0    1444
20080219.0    1432
20091201.0    1424
20090702.0    1402
              ... 
20140628.0       1
20030407.0       1
20060219.0       1
20130128.0       1
20190923.0       1
Name: 차량제작일자, Length: 8299, dtype: int64

In [78]:
temp = df.iloc[:, 9].fillna(0).astype('int')
temp[:5]

0    20110811
1    20061104
2    20091210
3    20070207
4    20081123
Name: 차량제작일자, dtype: int32

In [79]:
count = 0
idx9 = []
for i, one in tqdm(enumerate(temp.to_numpy())):
    if len(str(one)) != 8:
        idx9.append(i)
        count += 1
count

1528586it [00:01, 779094.54it/s]


169665

In [80]:
count = 0
idx91 = []
for i, one in tqdm(enumerate(temp.to_numpy())):
    if one == 0:
        idx91.append(i)
        count += 1
count

1528586it [00:00, 1530816.00it/s]


169661

In [81]:
len(list(set(idx9)^set(idx91)))

4

In [86]:
count = 0
p92 = re.compile('\d+')
idx92 = []
for i, one in tqdm(enumerate(temp.to_numpy())):
    if not p92.search(str(one)):
        idx92.append(i)
        count += 1
count

1528586it [00:02, 628080.16it/s]


0